In [4]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
load_dotenv()

True

## load variables
- data folder
- cleaned_df - for only halfmoons
- json template
- 

In [5]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
dff = pd.read_csv(cleaned_csv)
cleaned_df = dff[dff['halfmoons-halfmoon_details-halfmoon_type'].notnull()]
halfmoon_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'cleaned_trees_grasses_halfmoons.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")

with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



### get names from db
the cell has two queries:
- get all somalia users
- get user details for json population

In [6]:
# create a db engine
engine = create_engine(os.getenv('DATABASE_URL1'))

#query 
query = """select id, username, first_name, last_name, email, country from respi_regreeningusers where country in ('Somalia', 'somalia') order by first_name asc;"""
query2 = """select id, first_name, last_name, gender, age_category, phone_number, email, country, organization, username from respi_regreeningusers where id in(3228,1831,3726,4247,4548,2495);"""
#execute the query and fetch the results
conn = engine.connect()
df = pd.read_sql_query(query, conn)
df2 = pd.read_sql_query(query2, conn)

#close the connection
conn.close()
engine.dispose()

df2

,id,first_name,last_name,gender,age_category,phone_number,email,country,organization,username
0,1831,Muktar Mohamednoor,Rage,Male,18 – 35,252616051599,mrage@so-actionagainsthunger.org,Somalia,ACF,mrage
1,2495,Hassan Omar,Khadhib,Male,,0615807566,khadhiib10@gmail.com,Somalia,Norwegian Refugee Council,Khadhib
2,3228,Jamaal,Tagal,Male,18 – 35,252616719022,suldaanyare1265@gmail.com,Somalia,ACF,JamalAli
3,3726,Abdiaziz Adan,Hassan,Male,18 – 35,0615704046,abdulazizadan02@gmail.com,Somalia,GREDO,guudow
4,4247,Abdikarim,Mohamed,Male,18 – 35,252617144970,in.qooley@gmail.com,Somalia,GREDO SOMALIA,qoley
5,4548,Abdirashid,Sheikh Mohamed,,,0615197528,abdirashid.kassim@gredosom.org,Somalia,Gredo,NRMO


## enumerator
get user details from db for json population

In [7]:
for idx, i in enumerate(df2.to_dict(orient='records'), start=1):
    i['type'] = 'ENUMERATOR'
    ic(i)

ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'mrage@so-actionagainsthunger.org',
        'first_name': 'Muktar Mohamednoor',
        'gender': 'Male',
        'id': 1831,
        'last_name': 'Rage',
        'organization': 'ACF',
        'phone_number': '252616051599',
        'type': 'ENUMERATOR',
        'username': 'mrage'}
ic| i: {'age_category': '',
        'country': 'Somalia',
        'email': 'khadhiib10@gmail.com',
        'first_name': 'Hassan Omar',
        'gender': 'Male',
        'id': 2495,
        'last_name': 'Khadhib',
        'organization': 'Norwegian Refugee Council',
        'phone_number': '0615807566',
        'type': 'ENUMERATOR',
        'username': 'Khadhib'}
ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'suldaanyare1265@gmail.com',
        'first_name': 'Jamaal',
        'gender': 'Male',
        'id': 3228,
        'last_name': 'Tagal',
        'organization': 'ACF',
        'phone_nu

## farmingEntity
- read column values from cleaned df and append to json keys

In [8]:

#read and set values for farming entity
farming_entity = template[0]['farmingEntity'] 
#ic(farming_entity)

for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    ic(farming_entity)

ic| farming_entity: {'first_name': 'acf',
                     'id': 1,
                     'last_name': 'acf',
                     'middle_name': 'acf',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'acf',
                     'id': 2,
                     'last_name': 'acf',
                     'middle_name': 'acf',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'acf',
                     'id': 3,
                     'last_name': 'acf',
                     'middle_name': 'acf',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'nrc',
                     'id': 4,
                     'last_name': 'nrc',
                     'middle_name': 'nrc',
                     'organization': 'INSTITUTION'}
ic| farming_entity: {'first_name': 'nrc',
                     'id': 5,
                     'last_name': 'nrc',
                     'middle_name': 'nrc',
            

## plot-details
- read df to dictionary, for matching keys, extract df values
### plot-points
- parse the geoshape column from cleaned df
- split and extract the values from each i
- append to json keys

## crops
- get matching rows based on key(id)
- for these rows if crop has crops, get its values and 'other' if any
- clean 'and' from the crop types

In [9]:
import uuid
import copy
import re

def parse_geoshape(geoshape_str):
    #parse a geoshape string into a list of point dictionaries
    points = []
    parts = geoshape_str.split(';')

    for part in parts:
        part = part.strip()
        if not part:
            continue

        lat, lon, alt, acc = map(float, part.split(' '))

        points.append({
            "longitude": lon,
            "latitude": lat,
            "altitude": alt,
            "accuracy": acc
        })

    return points


final_list = []
point_id = 1

records = cleaned_df.to_dict(orient='records')

count = 0
#iterate over every i to capture plot details
for idx, i in enumerate(records):
    plot_id = idx + 1
    count += 1

    # ---- BUILD PLOT DETAILS ----
    template_cp = copy.deepcopy(template[0])
    plot_details = template_cp['plot']

    plot_details['name'] = uuid.uuid4().hex
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = "https://radrs.icraf.org/uploaded/media/" + i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']
    plot_details['livestock_allowed'] = False if i['land_use_selection-land_use'] == 'agriculture' else True
    plot_details['conservation_area'] = False

    #get only child row for this key
    matching_child_rows = halfmoon_df[halfmoon_df['PARENT_KEY'] == i['KEY']]
    
    
    """----------------------------------CROP STATUS ----------------------------------"""
    #iterate over matching child is to build crop status
    for x in matching_child_rows.to_dict(orient='records'):
        #plot_details['has_crops'] = True if x['crops_present'] == 'yes' else False
        #val = x['crops_in_field']
        if pd.isna(x['crops_in_field']):
            plot_details['has_crops'] = False
        else:
            plot_details['has_crops'] = True

        #get a list of all crops in field
        if pd.isna(x['crops_in_field']):
            crop_list = []
        else:
            crop_list = str(x['crops_in_field']).split(' ')

        #check that 'other' crops exist
        if 'other' in crop_list:
            other_crops_value = re.split(r'[,\s]+', x['other_crops'].strip())
            
            #get rid of 'and' if exists
            if 'and' in other_crops_value:
                other_crops_value.remove('and')

            #only add if not empty
            if other_crops_value:
                crop_list.remove('other')
                crop_list.extend(other_crops_value)
        
            #crop_list.remove('and')

        plot_details['crops'] = crop_list            

        
    """--------------------------------GEOSHAPE PARSING ----------------------------------"""
    geoshape_str = i['plot_details-polygon']
    parsed_points = parse_geoshape(geoshape_str)

    points_list = []
    for p in parsed_points:
        points_list.append({
            "id": point_id,
            "plotId": plot_id,
            "longitude": p["longitude"],
            "latitude": p["latitude"],
            "altitude": p["altitude"],
            "accuracy": p["accuracy"]
        })
        point_id += 1


    """--------------------------------ADMINISTRATIVE DETAILS ----------------------------------"""
    administrative = plot_details['subCounty']
    administrative['subcounty_name'] = i['geography-district_name']
    administrative['county']['county_name'] = i['geography-region_name']


    #update plot_details with number of points and administrative info
    plot_details['points'] = points_list
    plot_details['subCounty'] = administrative
    
    # del plot_details['points']
    # del plot_details['subCounty']

    final_list.append({
        "plot": plot_details
    })


In [10]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(final_list, f, indent=4)

str(halfmoon_df['crops_in_field'][20]).split(' ')

['maize', 'sorghum', 'beans', 'tomato']

## project

In [11]:
project = template[0]['project']
for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    project['id'] = idx
    project['project_name'] = i['land_use_selection-project_name']
    #ic(project)


## extract repeat csv records for cleaned data
1. get the 'KEY'(works like key and foreign key in the db) values from the main.csv to a list
2. For every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list
3. save the files

In [12]:
#step 1: get the 'KEY' values from cleaned_df to a list
keys = cleaned_df['KEY'].tolist()

#step 2.1: get the other csv filenames
dir = os.path.join(os.getcwd(),'data','brics_csvs')
main_csv = 'Rangeland_Somalia_BRCiSIII.csv'
#halfmoon_csv = 'Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv'
filenames = [file for file in os.listdir(dir) if file.endswith('.csv') and file != main_csv]

#step2.2 : for every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list

for name in filenames:
    #read the repeat csv
    doi = pd.read_csv(os.path.join(dir,name))

    #construct its new name
    new_filename = f'cleaned_{name}'

    #gey the filtered row values for mtching keys
    filtered_doi = doi[doi['PARENT_KEY'].isin(keys)]

    #step3: save the csvs
    filtered_doi.to_csv(os.path.join(dir,'outputs',new_filename), index=False)

    print(f'💯Created: {new_filename} 🎯with {len(filtered_doi)} records')


💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gully.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_contour.csv 🎯with 11 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_swales.csv 🎯with 62 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_keyline.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gabions.csv 🎯with 6 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_terracing.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_bunds.csv 🎯with 2 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv 🎯with 212 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_basins.csv 🎯with 0 records
💯Created: cleaned_cleaned_trees_grasses_halfmoons.csv 🎯with 212 records


# Microcatchments
- loop through the the records and extract microcatchment data.

In [ ]:

micros = []
for idx, c in enumerate(records):
    #use plot template copy, to be able to append here. 
    #microcatchment = template_cp['microcatchment'][0]
    microcatchment = copy.deepcopy(template[0])['microcatchment'][0]
    """---------------------------------------MICROCATCHMENT DETAILS---------------------------"""
    id = idx + 1
    microcatchment['id'] = id
    microcatchment['microcatchment_type'] = "Halfmoons"
    microcatchment['total_microcatchments'] = c['halfmoons-halfmoon_details-total_halfmoons']
    microcatchment['length'] = c['halfmoons-halfmoon_details-length_decimal']
    microcatchment['width'] = c['halfmoons-halfmoon_details-width_decimal']
    microcatchment['depth'] = c['halfmoons-halfmoon_details-depth_decimal']
    microcatchment['vertical_spacing'] = c['halfmoons-halfmoon_details_2-vertical_space']
    microcatchment['horizontal_spacing'] = c['halfmoons-halfmoon_details_2-horizontal_space']
    microcatchment['established_date'] = c['halfmoons-halfmoon_details-date']
    microcatchment['who_manages_microcatchment'] = ["MALE","FEMALE"] if c['halfmoons-halfmoon_details_2-manages'] == 'both' else ['MALE']
    if c['halfmoons-halfmoon_details_2-youth_manages'] == 'yes':
        microcatchment['who_manages_microcatchment'].append('YOUTH')

    '''--------------------------------------MANAGEMENT_PRACTICES--------------------------------'''
    #get only child rows for this record("KEY")    
    matching_mgmt_child_rows = halfmoon_df[halfmoon_df['PARENT_KEY'] == c['KEY']]
    # #check for matches
    # children = matching_mgmt_child_rows['PARENT_KEY'].tolist()
    # print(f"parent: ",c['KEY'], "child", len(children), children)

    practices = []
    child_records = matching_mgmt_child_rows.to_dict(orient='records')

    for  y in child_records:
        if y['additional_management'] == 'yes':
            vals = str(y['management_practices']).split(' ')
            practices.extend(vals)

    # make list a set - this removes duplicates/
    # the management practices in odk are halfmoon based,
    # in regreen app schema, they are microcatchment based
    # therefore we will be using unique management values only/
    set_practices = set(practices)
    #make all characters upper case(RA app schema requirement)
    ls_practices = [a.upper() for a in list(set_practices)]
    
    #dictionary to map odk values to RA app schema values
    mapping_dict = {
        "STICKS_BRANCH":"STICKS_PROTECTION",
        "DEBRIS_REMOVAL":"DEBRIS_REMOVAL",
        "INVASIVE_SPECIES":"INVASIVE_CONTROL",
        "WEEDING":"WEEDING",
        "WATERING":"WATERING",
        "PLANTING_SEEDLING":"PLANTING_ANNUAL_PERENNIAL_CROPS"
    }

    #map the odk values to accepted ra values
    normalised_practices = [v for k,v in mapping_dict.items() if k in ls_practices]
    microcatchment['management_practices'] = normalised_practices
    
    #for values not in ra, append them to other managements
    other_managements = [x for x in ls_practices if x not in mapping_dict.keys()]
    microcatchment['other_managements'] = other_managements

    microcatchment['key'] = c['KEY']

    """----------------------------------------CURRENT_STATUS-----------------------------------------"""

    cureent_status_list = []
    count = 0
    for j in child_records:
        #mapping dictionary for herbacious cover
        hb_cover_mapping = {
            1: "LESS_FOUR",
            2: "FOUR_FIFTEEN",
            3: "FIFTEEN_FOURTY",
            4: "FOURTY_SIXTFIVE",
            5: "SIXTHFIVE_ABOVE"
        }

        status = {
            "id": count + 1,
            "percentage_herbaceous_cover_in": hb_cover_mapping[j['herbaceous_cover']],
            "percentage_herbaceous_cover_between": hb_cover_mapping[j['herbaceous_cover_between']],
            "intact": True if j['halfmoon_intact'] == 'yes' else False,
            "protected": False,
            "protection_type": [],
            "other_protection_type": "",
            "visible_erosion": True if j['visible_erosion'] == 'yes' else False,
            "erosion_type_inside": [],
            "other_erosion_type_inside": "",
            "erosion_type_outside": [],
            "other_erosion_type_outside": "",
            "longitude": j['gps-Longitude'],
            "latitude": j['gps-Latitude'],
            "altitude": j['gps-Altitude'],
            "accuracy": j['gps-Accuracy'],
        }
        cureent_status_list.append(status)
        print(f"parent:{j['PARENT_KEY']}, child:{j['KEY']}")

    microcatchment['currentStatus'] = cureent_status_list

    micros.append(microcatchment)
    #break
    
    

parent:uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47, child:uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47/halfmoons/trees_grasses_halfmoons[1]
parent:uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47, child:uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47/halfmoons/trees_grasses_halfmoons[2]
parent:uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47, child:uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47/halfmoons/trees_grasses_halfmoons[3]
parent:uuid:366f2dfd-c43f-4b67-b8a2-b5056ad18b45, child:uuid:366f2dfd-c43f-4b67-b8a2-b5056ad18b45/halfmoons/trees_grasses_halfmoons[1]
parent:uuid:366f2dfd-c43f-4b67-b8a2-b5056ad18b45, child:uuid:366f2dfd-c43f-4b67-b8a2-b5056ad18b45/halfmoons/trees_grasses_halfmoons[2]
parent:uuid:366f2dfd-c43f-4b67-b8a2-b5056ad18b45, child:uuid:366f2dfd-c43f-4b67-b8a2-b5056ad18b45/halfmoons/trees_grasses_halfmoons[3]
parent:uuid:b4a2d545-66fe-4d23-825b-41025149b460, child:uuid:b4a2d545-66fe-4d23-825b-41025149b460/halfmoons/trees_grasses_halfmoons[1]
parent:uuid:b4a2d545-66fe-4d23-825b-41025149b460, child

In [18]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(micros, f, indent=4)


In [20]:
len(matching_child_rows)

4